<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3%E3%82%B5%E3%83%BC%E3%83%81%E5%88%A9%E6%B4%BB%E7%94%A8%E3%82%B9%E3%82%AD%E3%83%BC%E3%83%9E%E3%81%8B%E3%82%89%E3%82%B5%E3%83%A0%E3%83%8D%E3%82%A4%E3%83%AB%E7%94%BB%E5%83%8F%E3%81%A8%E3%82%B3%E3%83%B3%E3%83%86%E3%83%B3%E3%83%84%E3%81%AEURL%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ジャパンサーチ利活用スキーマからサムネイル画像とコンテンツのURLを取得するプログラム例

In [5]:
import json
import urllib.parse
import requests

###

# パラメータ

debug = True
unit = 1000
endpoint = "https://jpsearch.go.jp/rdf/sparql/"
output_file_path = "data.json"
provider = "chname:水研機構図書資料デジタルアーカイブ"

###

page = 0
data = {}

while 1:
    q = f"""
        PREFIX jps: <https://jpsearch.go.jp/term/property#>
        PREFIX schema: <http://schema.org/>
        PREFIX chname: <https://jpsearch.go.jp/entity/chname/>
        SELECT ?cho ?image ?associatedMedia WHERE {{
            ?cho schema:image ?image;
                jps:sourceInfo/schema:provider {provider} .
            optional {{
                ?cho jps:accessInfo/schema:associatedMedia ?associatedMedia .
            }}
        }} limit {unit} offset {unit * page}
    """

    url = endpoint+"?query="+urllib.parse.quote(q)+"&format=json&output=json&results=json"
    results = requests.get(url).json()
    if len(results["results"]["bindings"]) == 0:
        break

    page += 1

    if debug:
        print("page", page)

    for i in range(len(results["results"]["bindings"])):
        
        obj = results["results"]["bindings"][i]
        cho = obj["cho"]["value"]

        image = obj["image"]["value"]

        id = cho.split("/")[-1]

        item = {
            "image": [
                image
            ]
        }

        if "associatedMedia" in obj:
            associatedMedia = obj['associatedMedia']['value']
            item["associatedMedia"] = [
                associatedMedia
            ]

        data[id] = item

with open(output_file_path, 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4,
    sort_keys=True, separators=(',', ': '))


page 1
